# Wildfire risk by US county

## Introduction

### Obtain all zipcodes for the state

In [37]:
state_name = 'CA'
filename = 'data/zipcodes_'+state_name+'.csv'
import pandas as pd
counties = pd.read_csv(filename, index_col=[0],names=['zipcode','city','county'])
print("%d zipcodes found..." % len(counties))
with pd.option_context('display.max_rows',10) : 
    print(counties)

2589 zipcodes found...
                     city       county
zipcode                               
90001         Los Angeles  Los Angeles
90002         Los Angeles  Los Angeles
90003         Los Angeles  Los Angeles
90004         Los Angeles  Los Angeles
90005         Los Angeles  Los Angeles
...                   ...          ...
96157    South Lake Tahoe    El Dorado
96158    South Lake Tahoe    El Dorado
96160             Truckee       Nevada
96161             Truckee       Nevada
96162             Truckee       Nevada

[2589 rows x 2 columns]


### Load cencus data 

In [38]:
from uszipcode import SearchEngine

search = SearchEngine(simple_zipcode=True)
countydata = {}
skipped_zip=0
for zipcode in list(counties.index) : 
    data = search.by_zipcode(str(zipcode)).to_dict()
    try : 
        countydata[zipcode] = {
            "City":data["major_city"]+" "+data["state"],
            "Latitude":data["lat"],
            "Longitude":data["lng"],
            "Population":data["population"],
            "Density":int(data["population_density"]),
            "Occupancy":int(data["occupied_housing_units"] / data["housing_units"] * 100)/100,
            "Value":int(data["median_home_value"]/100)/10,
            "Income":int(data["median_household_income"]/100)/10
        }
    except:
        skipped_zip = skipped_zip+1
#         print("WARNING: error processing %s rbs, skipping" % (zipcode))
print("WARNING: missing data in %d out of %d zipcodes" % (skipped_zip,len(counties)))

### Load FEMA data

The data is retrieved from https://www.fema.gov/openfema-dataset-disaster-declarations-summaries-v2

In [41]:
fema_filename = 'data/DisasterDeclarationsSummaries.csv'
fire_data = pd.read_csv(fema_filename, index_col=[1],names=["femaDeclarationString",
                                                            "disasterNumber",
                                                            "state",
                                                            "declarationType",
                                                            "declarationDate",
                                                            "fyDeclared",
                                                            "incidentType",
                                                            "declarationTitle",
                                                            "ihProgramDeclared",
                                                            "iaProgramDeclared",
                                                            "paProgramDeclared",
                                                            "hmProgramDeclared",
                                                            "incidentBeginDate",
                                                            "incidentEndDate",
                                                            "disasterCloseoutDate",
                                                            "fipsStateCode",
                                                            "fipsCountyCode",
                                                            "placeCode",
                                                            "designatedArea",
                                                            "declarationRequestNumber",
                                                            "hash","lastRefresh"])
print('Types of incidents filed in the data set...')
print(set(fire_data['incidentType']))
# for item in list(fire_data.index) : 
#     print(item)
# with pd.option_context('display.max_rows',10) : 
#     print(fire_data)

Types of incidents filed in the data set...
{'Flood', 'Human Cause', 'Tsunami', 'Snow', 'Freezing', 'Severe Ice Storm', 'Dam/Levee Break', 'incidentType', 'Tornado', 'Chemical', 'Drought', 'Fire', 'Hurricane', 'Coastal Storm', 'Severe Storm(s)', 'Earthquake', 'Mud/Landslide', 'Fishing Losses', 'Volcano', 'Terrorist', 'Toxic Substances', 'Other', 'Typhoon'}


*Assuming only fire incidents are relevent to this study

### Getting California fire incident data

In [116]:
import requests 
from bs4 import BeautifulSoup
import re
table_content = []
final_list = []
list_of_avail_years = range(2013,2019,1)
for yr in list_of_avail_years : 
    url = 'https://www.fire.ca.gov/incidents/'+str(yr)+'/'
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    html_content = {}
    for i in range(len(soup.find_all('div'))) : 
        try : 
            if soup.find_all('div')[i].attrs['class'] == ['section', 'incidents-esri-webmap']: 
                html_content = soup.find_all('div')[i].text
        except : 
            continue
    
    table_content=eval(html_content.partition('create_map([')[2].split("\n")[0][:-3])
    for t in table_content : 
        final_list.append(t)
print(final_list)

[{'IncidentId': 653, 'UniqueId': '7a4a3552-3bea-4428-984c-e2edef13980f', 'Name': 'Becks Fire', 'Location': 'North of Highway 29, southeast of Kelseyville', 'Latitude': 0.0, 'Longitude': 0.0, 'AcresBurnedDisplay': '296', 'PercentContainedDisplay': '100%', 'CountiesList': 'Lake', 'UpdatedDate': '2013-01-23 10:15:00', 'StartedDate': '2013-01-22', 'AdminUnit': 'CAL FIRE Sonoma-Lake-Napa Unit', 'IncidentTypeDisplay': '', 'Url': 'https://www.fire.ca.gov/incidents/2013/1/22/becks-fire/', 'IsActive': 'N'}, {'IncidentId': 654, 'UniqueId': '094719ba-a47b-4abb-9ec5-a506b2b9fd23', 'Name': 'River Fire', 'Location': 'south of Narrow Gauge Rd & north of Hwy 136, east of Lone Pine', 'Latitude': 36.602575, 'Longitude': -118.01651, 'AcresBurnedDisplay': '406', 'PercentContainedDisplay': '100%', 'CountiesList': 'Inyo', 'UpdatedDate': '2013-02-28 20:00:00', 'StartedDate': '2013-02-24', 'AdminUnit': 'CAL FIRE San Bernardino Unit', 'IncidentTypeDisplay': '', 'Url': 'https://www.fire.ca.gov/incidents/2013/2/

In [118]:
fires_df = pd.DataFrame(final_list)

In [119]:
print(fires_df)

      IncidentId                              UniqueId           Name  \
0            653  7a4a3552-3bea-4428-984c-e2edef13980f     Becks Fire   
1            654  094719ba-a47b-4abb-9ec5-a506b2b9fd23     River Fire   
2            655  99cb21da-9639-4300-9a97-1cadc993189f    Jurupa Fire   
3            656  3513f584-632d-4fd0-a98a-646eb0753248     Butte Fire   
4            657  58f89ff8-bd3e-4355-b1c0-8fa05c747d3f  Fawnskin Fire   
...          ...                                   ...            ...   
1356        2009  d18da394-5ed4-4004-8dea-600e15d398e4    Sierra Fire   
1357        2010  03f49769-b928-44a5-be82-236d3d341eba     Niles Fire   
1358        2011  c003df4f-d19c-4e21-a90f-81d3f14aaf99    Briggs Fire   
1359        2012  c3b66e33-ef00-4874-a29b-b36640c16122      Peak Fire   
1360        2013  b6fb27e6-073c-4092-b8a8-51e96be987a4     Bear Fire    

                                               Location   Latitude  \
0         North of Highway 29, southeast of Kelseyvil